## EE 242 Lab 1a – Modifying Signals - Sampling and Amplifying

**Anna Petrbokova, Leonard Paya, Henry Adams, Grace Hwang, Team AA4**

This lab has 2 exercises to be completed as a team. Each should be given a separate code cell in your Notebook, followed by a markdown cell with report discussion. Your notebook should start with a markdown title and overview cell, which should be followed by an import cell that has the import statements for all assignments. For this assignment, you will need to import: numpy, the wavfile package from scipy.io, and matplotlib.pyplot.  

In [45]:
# We'll refer to this as the "import cell." Every module you import should be imported here.
import numpy as np
import matplotlib
import scipy.signal as sig
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt

# Specifically for displaying audio in the notebook
from IPython.display import Audio
# import whatever other modules you use in this lab -- there are more that you need than we've included 

## Summary

In this lab, you will work through a series of exercises to introduce you to working with audio signals and explore the impact of different amplitude and time operations on signals.  This is a two-week lab.  You should plan on completing the first 2 assignments in the first week.

## Lab 1a turn in checklist

•	Lab 1a Jupyter notebook with code for the 2 exercises assignment in separate cells. Each assignment cell should contain markdown cells (same as lab overview cells) for the responses to lab report questions. Include your lab members’ names at the top of the notebook.

**Please submit the report as PDF**




## Assignment 1 -- Working with sound files

Start a new cell following the guidelines in the Lab 1a template, dividing it into Parts A-C.

**A.**  Download the train.wav sound file provided. Read in the file saving the audio vector and sampling frequency in variables $x_1(t)$ and $f_{s_1}$, respectively. Print the sampling rate $f_{s_1}$ (which should be 32kHz) and the shape of $x_1(t)$, which will tell you the length and number of channels.  

**B.**  Write out two new versions of the file in wav format using different sampling rates: $f_{s_2}=f_{s_1}/2$ (16 kHz) and $f_{s_3}=1.5f_{s_1}$.  Basically create $x_2(t)=x_1(t/2)$ and $x_3(t)=x_1(2t)$

**C.**  Read in the three different versions of the train sound file and play each one.   ($x_1(t), x_2(t), x_3(t)$, don't change the $f_s$ while playing)

In [46]:
# Assignment 1 - Time Scaling Function
# Start with a comment section that explains what the input and output variables are, e.g.
# x: input signal vector
# fs: sampling rate (in Hz)
# a: scaling parameter. This has to be a decimal value for as_integer_ratio to work. 
# So, explicitly casting it into a float or a double or any fractional data type will help.
# returns t: time samples vector corresponding to y: scaled signal

def timescale(x, fs, a):
    n, d = (np.double(a)).as_integer_ratio()
    y = sig.resample_poly(x,d,n)
    t = np.arange(0,len(y),1)*(1.0/fs)
    return y,t


# TODO method comment
def linear_timescale(x, fs, a):
    n, d = (np.double(a)).as_integer_ratio() 
    num_samples = int(len(x) * d / n) 
    y = np.interp(np.linspace(0, len(x) - 1, num_samples),
        np.arange(len(x)), x)
    t = np.arange(0, len(y)) * (1.0 / fs)
    return y, t


In [47]:
# Assignment 1 - Playing and Plotting Time Scaled Audio Files 

# Part A
# 1. Read the train.wav file
samplerate, data = wav.read("train32.wav")

# convert to float32 format for the timescale operation
data = data.astype(np.float32)

# 2. Print the sampling rate and shape of signal
print(samplerate)
print(data.shape)

# Part B
# 1. Use both timescale and linear_timescale functions above to resample the signal using value of "a" as 0.5 and 2
fs2 = samplerate // 2
fs3 = 3 * fs2

sampled_timescale_high, times1 = timescale(data, fs3, 2)
sampled_timescale_low, times2 = timescale(data, fs2, 0.5)

sampled_linear_high, times1 = linear_timescale(data, fs3, 2)
sampled_linear_low, times2 = linear_timescale(data, fs2, 0.5)

# 2. Store the files as train_(high/low)_poly.wav and train_(high/low)_linear.wav (Don't change the f_s while storing the files)

wav.write('train_high_poly.wav', samplerate, sampled_timescale_high.astype(np.int16))
wav.write('train_low_poly.wav', samplerate, sampled_timescale_low.astype(np.int16))

wav.write('train_high_linear.wav', samplerate, sampled_linear_high.astype(np.int16))
wav.write('train_low_linear.wav', samplerate, sampled_linear_low.astype(np.int16))

# Part C
# 1. Play the files

fs, w = wav.read('train_low_poly.wav', )
display(Audio(w, rate=fs))

32000
(50313,)


###  Discussion

Comment on how the audio changes when the incorrect sampling frequency is used.

If the incorrect sampling frequency is used (for example, to play an existing audio file), then the resulting audio will play either faster or slower than it should. If the audio is slower, it will sound lower, whereas a more "compressed" sound will also be higher in pitch.


## Assignment 2 -- Amplitude Operations on Signals

Again, following the guidelines in the Lab 1 template, start a new cell and write a script to meet the following specifications. This assignment will have three parts, A-C, each of which should be indicated with comments.  

**A.1.** Create a discrete time signal $s_1(t)$ that is the same length as $x_1(t)$ and has value 1 for t=[0,0.5] and value 0.2 for $t>0.5$.  

You can use the command below where $len$ is the length of $x_1(t)$ and $n_0$ is the index corresponding to t=0.5, 
s1 = np.concatenate((np.ones(n0),0.2*np.ones(len(x1)-n0)) 

**A.2.** Multiply x1 with s1 to create v1. Save this signal to a wav file. 

**B.1** Create a discrete-time decaying ramp signal r1, that is the same length as x1. The signal should have value 1 at time 0 and linearly decay to value 0.  (Hint: use numpy.arange.) 

**B.2** Multiply x1 with r1 to create v2. Save this signal to a wav file.  

**C.**  Read in v1 and v2 and play the two different modifications together with the original, to verify that the volume of the second whistle is reduced. 

Ensure to play all the signals for the TAs

In [48]:
# Assignment 2 - Amplitude Operations on Signals

# Part A
# 1. Create a signal s1 which 1 from 0s to 0.5s and 0.2 from 0.5s onward
fs1, w1 = wav.read('train32.wav')
x1=w1
lengthx1 = len(x1)
n0 = int(0.5 * fs1)
s1 = np.concatenate((np.ones(n0), 0.2 * np.ones(len(x1)-n0)))

# 2. Multiply s1 with x1 sample by sample to create v1
v1= s1 * x1
#save v1 to a wav file
wav.write('v1.wav', fs1, v1.astype(np.int16))

# Part B
# 1. Create a signal r1 which goes down linearly from 1 to 0 across the length of the signal.
r1= np.linspace(1, 0, lengthx1)

# 2. Multiply r1 with x1 sample by sample to create v2
v2 = r1 * x1
wav.write('v2.wav', fs1, v2.astype(np.int16))

# Part C
# 1. Store and play v1 and v2
fs1, w1 = wav.read('v1.wav')
fs2, w2 = wav.read('v2.wav')

display(Audio(w1, rate=fs1))
display(Audio(w2, rate=fs2))


###  Discussion

Discuss the differences that the two modifications have on the signal. What would happen if you defined s1 to take value 2 for the [0,0.5] range? If you wanted a smooth but faster decay in amplitude, what signal might you use?  

If s1 is defined as suggested, then the signal would be louder than normal for the first half second, then quieter (exactly like how s1 is ordinarily defined on [0.5,1])

For a smooth, but faster decay in amplitude, one could consider using an exponential of the form a*e^{-bt}, where either a or b are greater than one. A quadratic such as (t-1)^2 would also suffice (on the unit interval).